<a href="https://colab.research.google.com/github/aditya-r-m/notes/blob/main/2RL_01_exploration_and_control.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Exploration** is the process of learning the environment structure better by trying out actions with potentially low known return, but a high degree of uncertainity.

**Exploitation** is the process of making actions which are known to be optimal to gather maximal cumulative reward.

### Multi-Armed Bandit

We have a set of distributions $\{R_a | a \in A\}$ where $A$ is known set of Actions (Arms). $R_a$ is a distribution on Rewards, given action $a$

The agent selects an action $A_t \in A$ at each timestep $t$ & the environment generates a reward $R_t \sim R_{A_t}$

The goal is to maximize cumulative reward $\sum_{i=1}^t R_i$

We do this by learning a policy, which is a distribution on $A$.

#### Valus & Regret

The action value for action $a$ is the expected reward $q(a) = E[R_t | A_t = a]$

The optimal value is $v_* = \max_{a \in A} q(a) = \max_a E[R_t | A_t = a]$

Regret of an action $a$ is $\Delta_a = v_* - q(a)$ which is zero for optimal action.

The problem of maximizing cumulative reward is identical to minizing total regret $L_t = \sum_{n=1}^t \Delta_{A_n}$ and Good policies will accumulate bounded regret.


#### Algorithms

There are several algorithms to approach the multi-armed bandit problem,

- Greedy
- $\epsilon$-Greedy
- UCB
- Thompson sampling
- Policy gradients

The first three all use action value estimates $Q_t(a) \approx q(a)$


#### Action Values estimation

A simple estimate for $q(a) = E[R_t | A_t = a]$ is the average of sampled rewards,

$
Q_t(a) = \frac{\sum_{n=1}^t I(A_n = a) R_n}{\sum_{n=1}^t I(A_n = a)}
$

where $I(\cdot)$ is the indicator function : $I(True) = 1$ and $I(False) = 0$ and the count for an action $a$ is

$
N_t(a) = \sum_{n=1}^t I(A_n = a)
$

This average can be learnt iteratively as,

$
Q_t(A_t)
= \left[1 - \frac{1}{N_t(A_t)}\right] Q_{t-1}(A_t) + \left[\frac{1}{N_t(A_t)}\right] R_t
= Q_{t-1} + \frac{1}{N_t(A_t)} \left[R_t - Q_{t-1}(A_t)\right]
$,

$
\forall a \neq A_t, Q_t(a) = Q_{t-1}(a)
$

where $N_0(a) = 0$

Note that the step-size $\alpha_t = \frac{1}{N_t(A_t)}$ which is mulitplied to the error term above leads to averaging, but other settings are also possible. For example, a constant step-size will lead to tracking instead of averaging - which may be used when the problem is non-stationary in some way.

#### Greedy and $\epsilon$-Greedy

Greedy approach can get stuck on a suboptimal actions, leading to linear expected total regret.

$\epsilon$-Greedy selects greedy action with probability $(1 - \epsilon)$ : $a = argmax_{a \in A} Q_t(a)$ and random action otherwise.

$
\pi_t(a) = \left\{
\begin{array}{ll}
      (1 - \epsilon) + \epsilon/|A| & \text{if} Q_t(a) = max_b Q_t(b) \\
        \epsilon/|A| & \text{otherwise} \\
\end{array}
\right.
$

$\epsilon$-greedy is a popular choice. Though with a constant $\epsilon$ it continues to explore, so it still leads to linear expected total regret.

#### Policy Search

We can also learn policies directly instead of learning values.
For instance, define action preferences $H_t(a)$ and a **softmax** policy,

$
\pi(a) = \frac{e^{H_t(a)}}{\sum_b e^{H_t(b)}}
$

These action preferences are not values, they are just learnable policy parameters.

Note that **softmax** is a generalization of **standard logistic function**
$f(x) = \frac{1}{1 + e^{-x}} = \frac{e^x}{1 + e^x}, \frac{d}{dx}f(x) = f(x) \cdot (1 - f(x))$

##### Policy gradients


In the bandit case, we want to update the current policy paramters $\theta_t$ as follows:

$
\theta_{t+1} = \theta_t + \alpha \nabla_\theta E[R_t | \pi_\theta]
$

$\theta_t$ can be thought of as a vector that contains just the action preferences. This can also be scaled to larger problems where $\theta_t$ represent parameters of deep neural networks.

The gradient of expectations is not directly known or computable - so we need some stochastic sample of this.

Note that this gradient is also not computable directly from samples of the reward distribution for a given action - as it depends on the softmax policy parameters.

Analyzing the error term $ \nabla_\theta E[R_t | \pi_\theta],
$ <br/> $
= \nabla_\theta \sum_a \pi_\theta(a) q(a)
$ <br/> $
= \sum_a q(a) \nabla_\theta \pi_\theta(a)
$ <br/> $
= E\left[\frac{R_t}{\pi_\theta(A_t)} \nabla_\theta \pi_\theta(A_t) \right]
= E\left[R_t \nabla_\theta log(\pi_\theta(A_t)) \right]
$


For softmax, we need to deal with two cases,
<br/> $
\frac{\delta}{\delta x}f(x, x)
= \frac{\delta}{\delta x} \frac{e^x}{e^x + \sum_{y \neq x} e^y}
= \frac{e^x \sum_{y \neq x} e^y}{\left(e^x + \sum_{y \neq x} e^y \right)^2}
= f(x, x) (1 - f(x, x))
\implies
\frac{\frac{\delta}{\delta x}f(x, x)}{f(x, x)} = 1 - f(x, x)
$ <br/> $
\frac{\delta}{\delta x}f(w, x)
= \frac{\delta}{\delta x} \frac{e^w}{e^x + \sum_{y \neq x} e^y}
= \frac{-e^w e^x}{\left(e^x + \sum_{y \neq x} e^y \right)^2}
= -f(w, x) f(x, x)
\implies
\frac{\frac{\delta}{\delta x}f(w, x)}{f(w, x)} = - f(x, x)
$

Using the above results, we can update the parameters $\theta = H_t$ as follows,
<br/> $
H_{t+1}(a) = H_{t}(a) + \alpha R_{t} (I(A_t = a) - \pi_t(a))
$

